In [1]:
def get_image_size(path):
    with Image.open(path) as img:
        return img.size  # (width, height)
    
def localize_objects_path(path): 
    objectBox = []
    box = []
    temp = []
    imgWidth, imgHeight = get_image_size(path)
    
    client = vision.ImageAnnotatorClient()
    with open(path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    # print(type(content))
    # print(content)
    objects = client.object_localization(image=image).localized_object_annotations

    # print(f"Number of objects found: {len(objects)}")
    for object_ in objects:
        # print(f"\n{object_.name} (confidence: {object_.score})")
        # print("Normalized bounding polygon vertices: ")
        for vertex in object_.bounding_poly.normalized_vertices:
            # print(f" - ({vertex.x}, {vertex.y})")
            pixel_x = int(vertex.x * imgWidth)
            pixel_y = int(vertex.y * imgHeight)
            # print(f" - ({pixel_x}, {pixel_y})")
            temp.append(pixel_x)
            temp.append(pixel_y)
        box.append(temp[0])
        box.append(temp[1])
        box.append(temp[4])
        box.append(temp[5])
        temp = []
        # print(box)
        object_info = {
            "name": object_.name,
            "confidence": object_.score,
            "box": box,
        }
        box = []
        objectBox.append(object_info)
        
    return objectBox

def obj_detection_path(path):
    objects = localize_objects_path(path)

    fileName = os.path.basename(path)
    with Image.open(path) as img:
        width, height = img.size
        result = {
            'fileName': fileName,
            'size': [width, height],
        }

        for i, obj in enumerate(objects):
            x1, y1, x2, y2 = obj['box']
            cropped_img = img.crop((x1, y1, x2, y2))
            obj['crop_img'] = cropped_img

    result['objects'] = objects
    return result
            
        

In [3]:
import os
from PIL import Image
from google.cloud import vision  

path = 'C:/Users/user/Desktop/새 폴더/Detection/test_imge/AAIRLLENSleekandSturdyInchComputerDeskPerfectforWorkandStudyMultiPurposeTableforWritingDiningandWorkstation.jpg'

ass = obj_detection_path(path)
for obj in ass['objects']:
    cimg = obj['crop_img']
    cimg.show()      
